In [1]:
from pathlib import Path
import sys
project_root = next((parent for parent in [Path.cwd()] + list(Path.cwd().parents) if (parent / "pyproject.toml").exists()), Path.cwd())
sys.path.append(str(project_root))

In [2]:
from google.cloud import bigquery

client = bigquery.Client(project="trelis-arc")

table_name = "trelis-arc.arc.shortest_ratio_2_5x_filtered_250"

In [6]:
create_final_table_query = f"""
CREATE OR REPLACE TABLE `{table_name}` AS

WITH training_tasks AS (
    SELECT DISTINCT task_id
    FROM `trelis-arc.arc.arc_task_ids`
    WHERE subset = "arc-agi-1/training"
),
-- Clean programs by collapsing multiple empty lines into single empty lines
programs_cleaned AS (
    SELECT 
        k.task_id,
        -- Clean code by collapsing multiple consecutive newlines into at most one empty line
        -- Pattern matches multiple consecutive newlines with optional whitespace
        REGEXP_REPLACE(k.code, r'\\n(\\s*\\n)+', '\\n\\n') as code,
        k.model,
        k.predicted_train_output,
        k.predicted_test_output,
        k.correct_train_input,
        k.correct_test_input
    FROM `trelis-arc.arc.king_programs_ext` k
    INNER JOIN training_tasks t ON k.task_id = t.task_id
    WHERE k.model != 'hodel-translated'
),
-- Calculate metrics and filter by grid size
programs_with_metrics AS (
    SELECT 
        task_id,
        code,
        model,
        predicted_train_output,
        predicted_test_output,
        correct_train_input,
        correct_test_input,
        LENGTH(code) as program_length,
        -- Check if all train inputs are correct
        (SELECT LOGICAL_AND(correct_val.element) 
         FROM UNNEST(correct_train_input.list) AS correct_val) as all_train_correct,
        -- Check if all test inputs are correct
        (SELECT LOGICAL_AND(correct_val.element) 
         FROM UNNEST(correct_test_input.list) AS correct_val) as all_test_correct,
        -- Count correct examples
        (SELECT COUNTIF(correct_val.element) 
         FROM UNNEST(correct_train_input.list) AS correct_val) + 
        (SELECT COUNTIF(correct_val.element) 
         FROM UNNEST(correct_test_input.list) AS correct_val) as total_correct,
        ARRAY_LENGTH(correct_train_input.list) + ARRAY_LENGTH(correct_test_input.list) as total_possible,
        -- Check grid sizes for train output
        (SELECT MAX(ARRAY_LENGTH(grid_2d.element.list)) 
         FROM UNNEST(predicted_train_output.list) AS grid_2d) as max_train_grid_height,
        (SELECT MAX(ARRAY_LENGTH(row_1d.element.list)) 
         FROM UNNEST(predicted_train_output.list) AS grid_2d,
              UNNEST(grid_2d.element.list) AS row_1d) as max_train_grid_width,
        -- Check grid sizes for test output
        (SELECT MAX(ARRAY_LENGTH(grid_2d.element.list)) 
         FROM UNNEST(predicted_test_output.list) AS grid_2d) as max_test_grid_height,
        (SELECT MAX(ARRAY_LENGTH(row_1d.element.list)) 
         FROM UNNEST(predicted_test_output.list) AS grid_2d,
              UNNEST(grid_2d.element.list) AS row_1d) as max_test_grid_width,
        -- Normalize code for deduplication (remove all whitespace, lowercase)
        LOWER(REGEXP_REPLACE(code, r'\\s+', '')) as normalized_code
    FROM programs_cleaned
),
-- Filter by grid size (40x40) and require at least one correct
programs_filtered AS (
    SELECT *,
        -- Calculate success rate for ranking
        SAFE_DIVIDE(total_correct, total_possible) as success_rate
    FROM programs_with_metrics
    WHERE max_train_grid_height <= 40 AND max_train_grid_width <= 40
      AND max_test_grid_height <= 40 AND max_test_grid_width <= 40
      AND total_correct > 0
),
-- Find shortest, most correct program per task
task_benchmarks AS (
    SELECT 
        task_id,
        MIN(program_length) as shortest_best_length
    FROM (
        SELECT 
            task_id,
            program_length,
            ROW_NUMBER() OVER (
                PARTITION BY task_id 
                ORDER BY success_rate DESC, program_length ASC, code ASC
            ) as rank
        FROM programs_filtered
    ) ranked
    WHERE rank = 1
    GROUP BY task_id
),
-- Filter programs to those within 2.5x the shortest best program per task
programs_length_filtered AS (
    SELECT p.*
    FROM programs_filtered p
    INNER JOIN task_benchmarks b ON p.task_id = b.task_id
    WHERE p.program_length <= 2.5 * b.shortest_best_length
),
-- Deduplicate programs (same normalized code + task_id)
programs_deduplicated AS (
    SELECT 
        task_id, code, model, predicted_train_output, predicted_test_output,
        correct_train_input, correct_test_input, program_length, success_rate,
        ROW_NUMBER() OVER (
            PARTITION BY task_id, normalized_code
            ORDER BY success_rate DESC, program_length ASC, model ASC, code ASC
        ) as dedup_rank
    FROM programs_length_filtered
),
-- Take top 500 per task, prioritizing correctness then length
final_selection AS (
    SELECT 
        task_id, code, model, predicted_train_output, predicted_test_output,
        correct_train_input, correct_test_input,
        ROW_NUMBER() OVER (
            PARTITION BY task_id 
            ORDER BY success_rate DESC, program_length ASC, model ASC, code ASC
        ) as final_rank
    FROM programs_deduplicated
    WHERE dedup_rank = 1
)
SELECT task_id, code, model, predicted_train_output, predicted_test_output,
       correct_train_input, correct_test_input
FROM final_selection
WHERE final_rank <= 250
ORDER BY task_id, final_rank
"""

print("Executing BigQuery table creation...")
job = client.query(create_final_table_query)
result = job.result()
print(f"✓ Table `{table_name}` created successfully")

Executing BigQuery table creation...
✓ Table `trelis-arc.arc.shortest_ratio_2_5x_filtered_250` created successfully
✓ Table `trelis-arc.arc.shortest_ratio_2_5x_filtered_250` created successfully


In [ ]:
from llm_python.datasets.bigquery_export import load_bigquery_table_as_dataframe

# Load BigQuery table as DataFrame using our reusable function
print("Loading BigQuery table data...")
raw_data = load_bigquery_table_as_dataframe(
    client=client,
    table_name=table_name
)
print(f"Loaded {len(raw_data)} programs from BigQuery table")

Loading BigQuery table data...
Exporting BigQuery table 'trelis-arc.arc.shortest_ratio_2_5x_filtered_250' to GCS...
Waiting for BigQuery export to complete...
Waiting for BigQuery export to complete...
✓ Export to GCS completed successfully
✓ Export to GCS completed successfully
✓ Download completed
Reading parquet file...
✓ Download completed
Reading parquet file...
Loaded 59170 rows from BigQuery table
Loaded 59170 programs from BigQuery table
Loaded 59170 rows from BigQuery table
Loaded 59170 programs from BigQuery table


In [4]:
from llm_python.datasets import convert_bigquery_to_soar, save_soar_parquet

# First, let's inspect the actual data structure
print("Inspecting BigQuery data structure...")
sample_row = raw_data.iloc[0]
print(f"Sample row columns: {sample_row.index.tolist()}")
print(f"Train output type: {type(sample_row['predicted_train_output'])}")
print(f"Train correct type: {type(sample_row['correct_train_input'])}")

print("\n" + "="*50)

# Convert BigQuery data to SOAR format using our reusable function
print("Converting BigQuery data to SOAR format...")
final_dataset = convert_bigquery_to_soar(raw_data, show_progress=True)

# Save the final dataset
if len(final_dataset) > 0:
    file_name = table_name.split('.')[-1]  # Extract the last part of the table name
    output_path = f"/tmp/{file_name}.parquet"
    print(f"Saving final dataset to: {output_path}")
    
    save_soar_parquet(final_dataset, output_path)
else:
    print("No valid data to save!")

Inspecting BigQuery data structure...
Sample row columns: ['task_id', 'code', 'model', 'predicted_train_output', 'predicted_test_output', 'correct_train_input', 'correct_test_input']
Train output type: <class 'dict'>
Train correct type: <class 'dict'>

Converting BigQuery data to SOAR format...


Converting BQ to SOAR: 100%|██████████| 59170/59170 [00:06<00:00, 9365.96it/s] 



Successfully converted 59170 programs from 59170 input rows
Saving final dataset to: /tmp/shortest_ratio_2_5x_filtered_250.parquet
✓ Saved 59170 programs to /tmp/shortest_ratio_2_5x_filtered_250.parquet with proper PyArrow schema
✓ Saved 59170 programs to /tmp/shortest_ratio_2_5x_filtered_250.parquet with proper PyArrow schema


In [5]:
# Comprehensive verification of the final dataset
import duckdb
import pandas as pd

print("=" * 80)
print("COMPREHENSIVE DATASET VERIFICATION")
print("=" * 80)

file_path = f"/tmp/{file_name}.parquet"

# 1. Basic file and structure verification
print("\n1. BASIC DATASET INFO:")
print("-" * 40)
verification_df = pd.read_parquet(file_path)
print(f"✓ File loaded successfully")
print(f"✓ Dataset shape: {verification_df.shape}")
print(f"✓ Columns: {list(verification_df.columns)}")
print(f"✓ Unique tasks: {verification_df['task_id'].nunique()}")
print(f"✓ Programs per task (mean): {verification_df.groupby('task_id').size().mean():.2f}")
print(f"✓ Programs per task (max): {verification_df.groupby('task_id').size().max()}")
print(f"✓ Programs per task (min): {verification_df.groupby('task_id').size().min()}")

# 2. Data type verification
print("\n2. DATA TYPE VERIFICATION:")
print("-" * 40)
sample = verification_df.iloc[0]

# Check predicted outputs (should be 3D arrays: List[List[List[int]]])
train_output = sample['predicted_train_output']
test_output = sample['predicted_test_output']

print(f"✓ Train output type: {type(train_output)} (expected: list)")
print(f"✓ Train output length: {len(train_output)} grids")
if len(train_output) > 0:
    first_grid = train_output[0]
    print(f"✓ First grid type: {type(first_grid)} (expected: list)")
    if isinstance(first_grid, list) and len(first_grid) > 0:
        print(f"✓ Grid dimensions: {len(first_grid)} x {len(first_grid[0])}")
        if len(first_grid[0]) > 0:
            cell_value = first_grid[0][0]
            print(f"✓ Cell value type: {type(cell_value)} = {cell_value} (expected: int)")

print(f"✓ Test output type: {type(test_output)} (expected: list)")
print(f"✓ Test output length: {len(test_output)} grids")

# Check correctness arrays (should be 1D boolean arrays: List[bool])
train_correct = sample['correct_train_input']
test_correct = sample['correct_test_input']

print(f"✓ Train correct type: {type(train_correct)} (expected: list)")
print(f"✓ Train correct values: {train_correct}")
if len(train_correct) > 0:
    print(f"✓ First correct value type: {type(train_correct[0])} (expected: bool)")

print(f"✓ Test correct type: {type(test_correct)} (expected: list)")
print(f"✓ Test correct values: {test_correct}")

# 3. Data completeness verification
print("\n3. DATA COMPLETENESS:")
print("-" * 40)
non_empty_train = verification_df['predicted_train_output'].apply(lambda x: len(x) > 0).sum()
non_empty_test = verification_df['predicted_test_output'].apply(lambda x: len(x) > 0).sum()
non_empty_train_correct = verification_df['correct_train_input'].apply(lambda x: len(x) > 0).sum()
non_empty_test_correct = verification_df['correct_test_input'].apply(lambda x: len(x) > 0).sum()

print(f"✓ Rows with train outputs: {non_empty_train}/{len(verification_df)} ({100*non_empty_train/len(verification_df):.1f}%)")
print(f"✓ Rows with test outputs: {non_empty_test}/{len(verification_df)} ({100*non_empty_test/len(verification_df):.1f}%)")
print(f"✓ Rows with train correctness: {non_empty_train_correct}/{len(verification_df)} ({100*non_empty_train_correct/len(verification_df):.1f}%)")
print(f"✓ Rows with test correctness: {non_empty_test_correct}/{len(verification_df)} ({100*non_empty_test_correct/len(verification_df):.1f}%)")

# 4. Code cleaning verification
print("\n4. CODE CLEANING VERIFICATION:")
print("-" * 40)
# Check if codes have whitespace-only lines removed
codes_with_empty_lines = 0
total_codes_checked = min(100, len(verification_df))  # Check first 100

for i in range(total_codes_checked):
    code = verification_df.iloc[i]['code']
    lines = code.split('\n')
    for line in lines:
        if line.strip() == '':
            codes_with_empty_lines += 1
            break

print(f"✓ Codes with empty lines: {codes_with_empty_lines}/{total_codes_checked} (should be 0)")

# 5. DuckDB compatibility verification
print("\n5. DUCKDB COMPATIBILITY:")
print("-" * 40)
con = duckdb.connect()

try:
    # Schema check
    schema = con.execute(f"DESCRIBE '{file_path}'").fetchdf()
    print(f"✓ DuckDB can read schema ({len(schema)} columns)")
    
    # Basic query check
    sample_query = f"""
    SELECT 
        task_id,
        model,
        length(predicted_train_output) as num_train_grids,
        length(predicted_test_output) as num_test_grids,
        length(correct_train_input) as num_train_examples,
        length(correct_test_input) as num_test_examples
    FROM '{file_path}' 
    LIMIT 3
    """
    sample_data = con.execute(sample_query).fetchdf()
    print(f"✓ Basic queries work")
    print(f"  Sample data shape: {sample_data.shape}")
    
    # 3D array access check
    nested_query = f"""
    SELECT 
        task_id,
        predicted_train_output[1] as first_train_grid,
        length(predicted_train_output[1]) as grid_height,
        length(predicted_train_output[1][1]) as grid_width
    FROM '{file_path}' 
    WHERE length(predicted_train_output) > 0 
      AND length(predicted_train_output[1]) > 0
    LIMIT 2
    """
    nested_data = con.execute(nested_query).fetchdf()
    print(f"✓ 3D array access works")
    print(f"  Grid access sample: {nested_data.shape}")
    
    print(f"✓ All DuckDB operations successful!")
    
except Exception as e:
    print(f"✗ DuckDB error: {e}")

finally:
    con.close()

# 6. Summary statistics
print("\n6. DATASET STATISTICS:")
print("-" * 40)
task_stats = verification_df.groupby('task_id').size()
model_stats = verification_df['model'].value_counts()

print(f"✓ Tasks with programs: {len(task_stats)}")
print(f"✓ Programs per task: min={task_stats.min()}, max={task_stats.max()}, mean={task_stats.mean():.1f}")
print(f"✓ Models represented: {len(model_stats)}")
print(f"✓ Top 3 models: {dict(model_stats.head(3))}")

print("\n" + "=" * 80)
print("✅ VERIFICATION COMPLETE - Filtered dataset is ready for use!")
print("=" * 80)
print(f"📁 Final dataset location: {file_path}")
print(f"📊 Total programs: {len(verification_df):,}")
print(f"🎯 Unique tasks: {verification_df['task_id'].nunique()}")
print(f"🤖 Models: {len(model_stats)}")
print(f"📏 Max programs per task: {task_stats.max()} (target: ≤500)")
print("=" * 80)

COMPREHENSIVE DATASET VERIFICATION

1. BASIC DATASET INFO:
----------------------------------------
✓ File loaded successfully
✓ Dataset shape: (59170, 11)
✓ Columns: ['task_id', 'reasoning', 'code', 'correct_train_input', 'correct_test_input', 'predicted_train_output', 'predicted_test_output', 'train_input', 'test_input', 'model', 'generation']
✓ Unique tasks: 399
✓ Programs per task (mean): 148.30
✓ Programs per task (max): 250
✓ Programs per task (min): 2

2. DATA TYPE VERIFICATION:
----------------------------------------
✓ Train output type: <class 'numpy.ndarray'> (expected: list)
✓ Train output length: 5 grids
✓ First grid type: <class 'numpy.ndarray'> (expected: list)
✓ Test output type: <class 'numpy.ndarray'> (expected: list)
✓ Test output length: 1 grids
✓ Train correct type: <class 'numpy.ndarray'> (expected: list)
✓ Train correct values: [ True  True  True  True  True]
✓ First correct value type: <class 'numpy.bool'> (expected: bool)
✓ Test correct type: <class 'numpy.ndar

In [7]:
import duckdb

con = duckdb.connect()
sample_df = con.execute(f"SELECT * FROM '{file_path}' LIMIT 10").fetchdf()
con.close()
print(sample_df)

    task_id reasoning                                               code  \
0  007bbfb7            def transform(grid):\n    pattern = [[0] * 9 f...   
1  007bbfb7            def transform(grid):\n    output = [[0] * 9 fo...   
2  007bbfb7            def transform(grid):\n\n    output = [[0 for _...   
3  007bbfb7            def transform(input_grid):\n    output_grid = ...   
4  007bbfb7            import numpy as np\n\ndef transform(grid_lst: ...   
5  007bbfb7            def transform(grid):\n    n = len(grid)\n    m...   
6  007bbfb7            def transform(grid):\n    output_size = 9\n   ...   
7  007bbfb7            def transform(grid):\n\n    def expand_grid(gr...   
8  007bbfb7            def transform(grid):\n    n = len(grid)\n    n...   
9  007bbfb7            def transform(grid):\n    n = len(grid)\n    n...   

              correct_train_input correct_test_input  \
0  [True, True, True, True, True]             [True]   
1  [True, True, True, True, True]             [True